#Dataset

In [1]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

path = "FULL"
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


#VGG16

In [7]:
VGG16=tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG16.trainable = False
print(len(VGG16.layers))

19


In [8]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

In [9]:
input = tf.keras.layers.Input(shape=(224, 224, 3))
x = data_augmentation(input)
x = VGG16(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG16 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG16")

VGG16.summary()

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 average_pooling2d_1 (Averag  (None, 4, 4, 512)        0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 7)                 57351     
                                                                 
Total params: 14,772,039
Trainable params: 57,351
Non-trainable params: 14,714,688
____________________________________________

In [10]:
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = VGG16.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=100,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/100
31/31 [==============================] - 221s 7s/step - loss: 1.8552 - accuracy: 0.2601 - val_loss: 1.7940 - val_accuracy: 0.2835
Epoch 2/100
31/31 [==============================] - 224s 7s/step - loss: 1.7391 - accuracy: 0.3085 - val_loss: 1.6735 - val_accuracy: 0.3616
Epoch 3/100
31/31 [==============================] - 224s 7s/step - loss: 1.6458 - accuracy: 0.3684 - val_loss: 1.7166 - val_accuracy: 0.3103
Epoch 4/100
31/31 [==============================] - 222s 7s/step - loss: 1.5728 - accuracy: 0.3886 - val_loss: 1.5784 - val_accuracy: 0.4040
Epoch 5/100
31/31 [==============================] - 226s 7s/step - loss: 1.5442 - accuracy: 0.4178 - val_loss: 1.5111 - val_accuracy: 0.4509
Epoch 6/100
31/31 [==============================] - 221s 7s/step - loss: 1.5012 - accuracy: 0.4435 - val_loss: 1.4869 - val_accuracy: 0.4263
Epoch 7/100
31/31 [==============================] - 227s 7s/step - loss: 1.5164 - accuracy: 0.4229 - val_loss: 1.5321 - val_accuracy: 0.4263
Epoch 

In [11]:
VGG16.save('VGG16Augmented_Model_FullFreeze.keras')

In [5]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_FullFreeze.keras')
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = VGG16.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=250,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )
VGG16.save('VGG16Augmented_Model_FullFreeze.keras')

Epoch 1/250
31/31 [==============================] - 214s 7s/step - loss: 1.1469 - accuracy: 0.5801 - val_loss: 1.2542 - val_accuracy: 0.5446
Epoch 2/250
31/31 [==============================] - 210s 7s/step - loss: 1.0725 - accuracy: 0.6129 - val_loss: 1.2273 - val_accuracy: 0.5469
Epoch 3/250
31/31 [==============================] - 200s 6s/step - loss: 1.1142 - accuracy: 0.5968 - val_loss: 1.4045 - val_accuracy: 0.4978
Epoch 4/250
31/31 [==============================] - 230s 7s/step - loss: 1.0993 - accuracy: 0.5978 - val_loss: 1.3064 - val_accuracy: 0.5246
Epoch 5/250
31/31 [==============================] - 205s 7s/step - loss: 1.1000 - accuracy: 0.5968 - val_loss: 1.3317 - val_accuracy: 0.5000
Epoch 6/250
31/31 [==============================] - 211s 7s/step - loss: 1.1436 - accuracy: 0.5748 - val_loss: 1.3721 - val_accuracy: 0.5045
Epoch 7/250
31/31 [==============================] - 207s 7s/step - loss: 1.0996 - accuracy: 0.5892 - val_loss: 1.2666 - val_accuracy: 0.5089
Epoch 

KeyboardInterrupt: 

In [11]:
VGG16.save('VGG16Augmented_Model_FullFreeze.keras')

In [2]:
VGG16 = tf.keras.models.load_model('VGG16Augmented_Model_FullFreeze.keras')
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = VGG16.fit(x=training_set,
                    epochs=110,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )
VGG16.save('VGG16Augmented_Model_FullFreeze.keras')

Epoch 1/110
31/31 [==============================] - 179s 6s/step - loss: 0.9276 - accuracy: 0.6754
Epoch 2/110
31/31 [==============================] - 175s 6s/step - loss: 0.9034 - accuracy: 0.6749
Epoch 3/110
31/31 [==============================] - 176s 6s/step - loss: 0.9272 - accuracy: 0.6668
Epoch 4/110
31/31 [==============================] - 175s 6s/step - loss: 0.9188 - accuracy: 0.6749
Epoch 5/110
31/31 [==============================] - 176s 6s/step - loss: 0.9231 - accuracy: 0.6618
Epoch 6/110
31/31 [==============================] - 182s 6s/step - loss: 0.8882 - accuracy: 0.6875
Epoch 7/110
31/31 [==============================] - 176s 6s/step - loss: 0.8894 - accuracy: 0.6850
Epoch 8/110
31/31 [==============================] - 180s 6s/step - loss: 0.9317 - accuracy: 0.6709
Epoch 9/110
31/31 [==============================] - 177s 6s/step - loss: 0.9590 - accuracy: 0.6547
Epoch 10/110
31/31 [==============================] - 177s 6s/step - loss: 0.9293 - accuracy: 0.6704